In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sdgym
from sdgym.synthesizers import UniformSynthesizer, DataIdentity
from sklearn.preprocessing import LabelEncoder
import sys
sys.path.append('..')
from ctabganplus.model.evaluation import get_utility_metrics

In [2]:
unf_syn = UniformSynthesizer()
data_id = DataIdentity()

### Get the benchmarking baseline

In [3]:
from sdgym import load_dataset

In [4]:
train_data, metadata = load_dataset(modality="single-table", dataset='bm_datasets/train_data/')

In [5]:
hpo_data, metadata = load_dataset(modality="single-table", dataset='bm_datasets/hpo_data/')

In [6]:
train_model_unf = unf_syn.get_trained_synthesizer(train_data, metadata)
train_sample_data_unf = unf_syn.sample_from_synthesizer(train_model_unf, train_data.shape[0])
train_model_id = data_id.get_trained_synthesizer(train_data, metadata)
train_sample_data_ind = data_id.sample_from_synthesizer(train_model_id, train_data.shape[0])

In [7]:
hpo_model_unf = unf_syn.get_trained_synthesizer(hpo_data, metadata)
hpo_sample_data_unf = unf_syn.sample_from_synthesizer(hpo_model_unf, hpo_data.shape[0])
hpo_model_id = data_id.get_trained_synthesizer(hpo_data, metadata)
hpo_sample_data_id = data_id.sample_from_synthesizer(hpo_model_id, hpo_data.shape[0])

### Get Baseline Metrics

In [8]:
test_data = pd.read_csv('bm_datasets/test_data/test_data.csv')

In [9]:
hpo_data = pd.read_csv('bm_datasets/hpo_data/hpo_data.csv')

In [10]:
real_data = pd.read_csv('bm_datasets/train_data/train_data.csv')

In [11]:
le_dict = {"attack_type": "le_attack_type", "label": "le_label", "proto": "le_proto", "tos": "le_tos"}
for c in le_dict.keys():
    le_dict[c] = LabelEncoder()
    test_data[c] = le_dict[c].fit_transform(test_data[c])
    hpo_data[c] = le_dict[c].fit_transform(hpo_data[c])
    train_sample_data_unf[c] = le_dict[c].fit_transform(train_sample_data_unf[c])
    train_sample_data_ind[c] = le_dict[c].fit_transform(train_sample_data_ind[c])
    hpo_sample_data_unf[c] = le_dict[c].fit_transform(hpo_sample_data_unf[c])
    hpo_sample_data_id[c] = le_dict[c].fit_transform(hpo_sample_data_id[c])
    real_data[c] = le_dict[c].fit_transform(real_data[c])

In [ ]:
bm_id_results, cr_id = get_utility_metrics(hpo_data, test_data, hpo_sample_data_id, scaler="MinMax",type={"Classification":["xgb","lr","dt","rf","mlp"]})

In [23]:
bm_id_results.drop(["Model"],axis=1).groupby(["Type"]).mean().sort_values(by="F1_Score", ascending=False).head(100)

,Acc,AUC,F1_Score,SE_Acc,SE_AUC,SE_F1
Type,,,,,,
Fake,65.516171,0.874539,0.612798,0.082403,0.000549,0.000835
Real,61.350125,0.851963,0.560196,0.081251,0.000562,0.000786
Difference,-4.166045,-0.022576,-0.052602,-0.001151,0.000013,-0.000049


In [12]:
bm_unf_results, cr_unf = get_utility_metrics(hpo_data, test_data, hpo_sample_data_unf, scaler="MinMax",type={"Classification":["xgb","lr","dt","rf","mlp"]})

Model:  xgb trained on real data
Model:  lr trained on real data
Model:  dt trained on real data
Model:  rf trained on real data
Model:  mlp trained on real data
Model:  xgb trained on fake data
Model:  lr trained on fake data
Model:  dt trained on fake data
Model:  rf trained on fake data
Model:  mlp trained on fake data


In [16]:
bm_unf_results.drop(["Model"],axis=1).groupby(["Type"]).mean().sort_values(by="F1_Score", ascending=False).head(100)

,Acc,AUC,F1_Score,SE_Acc,SE_AUC,SE_F1
Type,,,,,,
Real,61.350125,0.851963,0.560196,0.081251,0.000562,0.000786
Difference,37.028260,0.348872,0.448018,0.008280,-0.000302,0.000256
Fake,24.321865,0.503091,0.112178,0.072971,0.000864,0.000530


In [13]:
# get the brute force and ping scan data from test_2 
bf = test_data[test_data['attack_type'] == 'bruteForce']
ps = test_data[test_data['attack_type'] == 'pingScan']
hpo_data = pd.concat([hpo_data, bf, ps])
hpo_data.reset_index(drop=True, inplace=True)

In [14]:
hpo_data.attack_type.value_counts()

0    253721
2     64172
4      1983
1       169
3         5
Name: attack_type, dtype: int64

In [15]:
train_id_results, cr_id_train = get_utility_metrics(real_data, test_data, train_sample_data_ind, scaler="MinMax",type={"Classification":["xgb","lr","dt","rf","mlp"]})

Model:  xgb trained on real data
Model:  lr trained on real data
Model:  dt trained on real data
Model:  rf trained on real data
Model:  mlp trained on real data
Model:  xgb trained on fake data
Model:  lr trained on fake data
Model:  dt trained on fake data
Model:  rf trained on fake data
Model:  mlp trained on fake data


In [17]:
train_id_results.drop(["Model"],axis=1).groupby(["Type"]).mean().sort_values(by="F1_Score", ascending=False).head(100)

,Acc,AUC,F1_Score,SE_Acc,SE_AUC,SE_F1
Type,,,,,,
Fake,86.709319,0.959889,0.784177,0.057144,0.000332,0.000668
Real,83.348174,0.936337,0.735378,0.059556,0.000372,0.000633
Difference,-3.361145,-0.023552,-0.048798,0.002412,0.000041,-0.000035


In [18]:
train_unf_results, cr_unf_train = get_utility_metrics(real_data, test_data, train_sample_data_unf, scaler="MinMax",type={"Classification":["xgb","lr","dt","rf","mlp"]})

Model:  xgb trained on real data
Model:  lr trained on real data
Model:  dt trained on real data
Model:  rf trained on real data
Model:  mlp trained on real data
Model:  xgb trained on fake data
Model:  lr trained on fake data
Model:  dt trained on fake data
Model:  rf trained on fake data
Model:  mlp trained on fake data


In [19]:
train_unf_results.drop(["Model"],axis=1).groupby(["Type"]).mean().sort_values(by="F1_Score", ascending=False).head(100)

,Acc,AUC,F1_Score,SE_Acc,SE_AUC,SE_F1
Type,,,,,,
Real,83.348174,0.936337,0.735378,0.059556,0.000372,0.000633
Difference,63.045393,0.424436,0.634904,-0.010440,-0.000497,0.000140
Fake,20.302780,0.511901,0.100475,0.069996,0.000870,0.000492
